In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import warnings
import holidays

from sklearn.preprocessing import LabelEncoder

warnings.filterwarnings(action='ignore') 

In [9]:
path = f'./processed_data/train_merge.csv'
df = pd.read_csv(path)
df = df[df['item']=='BC']
df[~df["Date"].isin(kr_holidays)]

for i in qual_col:
    le = LabelEncoder()
    df[i]=le.fit_transform(df[i])

name_lst = df.corr()['price(원/kg)'].sort_values().keys().to_list()
value_lst = df.corr()['price(원/kg)'].sort_values().values

print([a for a, b in zip(name_lst, value_lst) if abs(b)>0.15 and a != 'supply(kg)_x' ])

['location', 'week', 'supply(kg)_y', 'price(원/kg)']


In [38]:
# 필요 정보
# item_lst : item 리스트 항목
# qual_col : one-hot-encoder 변환이 필요한 항목

item_lst = ['TG', 'CR', 'CB', 'RD', 'BC']
qual_col = ['item', 'corporation', 'location','day_name']
columns_dict = {
    'TG' : ['year', 'month', 'day', 'location', 'corporation', 'supply(kg)_y', '수출 금액', '무역수지', '전체 거래량', '수출 중량', 'price(원/kg)'],
    'CR' : ['location', 'week', 'month', 'corporation', 'year', 'supply(kg)_y', 'price(원/kg)'],
    'CB' : ['month', 'week', 'location', '무역수지', '수출 금액', '수출 중량', 'corporation', 'supply(kg)_y', 'price(원/kg)'],
    'RD' : ['week', 'month', 'location', 'supply(kg)_y', 'price(원/kg)'],
    'BC' : ['location', 'week', 'supply(kg)_y', 'price(원/kg)']
}

# item 선택
item = item_lst[0]
path = f'./processed_data/{item}_train_merge.csv'

# 한국 2019~2023 공휴일 정보
kr_holidays = holidays.KR(years=[2019,2020,2021,2022,2023])
kr_holidays = [str(i) for i in kr_holidays]

In [39]:
train = pd.read_csv(path)

# 데이터 전처리 프로세스
train = train[~train["Date"].isin(kr_holidays)] # Holyday 제거

for i in qual_col:
    le = LabelEncoder()
    df[i]=le.fit_transform(df[i])

train = train[columns_dict[item]]

In [40]:
from pycaret.regression import *

s = setup(train, target = 'price(원/kg)', 
        session_id = 41, 
        fold = 10, 
        normalize = True, 
        normalize_method = 'minmax',
        train_size=0.8)

TG_reg = compare_models()

,Description,Value
0,Session id,41
1,Target,price(원/kg)
2,Target type,Regression
3,Original data shape,"(14460, 9)"
4,Transformed data shape,"(14460, 13)"
5,Transformed train set shape,"(11568, 13)"
6,Transformed test set shape,"(2892, 13)"
7,Ordinal features,1
8,Numeric features,6
9,Categorical features,2


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
xgboost,Extreme Gradient Boosting,1165.6034,3215798.2000,1792.6603,0.5843,3.4575,0.2047,0.0330
catboost,CatBoost Regressor,1285.5337,3642080.1778,1907.8337,0.5296,3.5888,0.2051,0.3470
lightgbm,Light Gradient Boosting Machine,1403.6961,4267496.8290,2065.1364,0.4488,3.6590,0.2100,0.2600
rf,Random Forest Regressor,1374.2293,4585393.3847,2140.5302,0.4074,3.5507,0.2171,0.2730
gbr,Gradient Boosting Regressor,1551.5562,4885439.0940,2209.6465,0.3693,3.7273,0.2279,0.1350
ada,AdaBoost Regressor,1780.0913,5573902.8008,2360.3822,0.2803,3.7642,0.2937,0.0690
ridge,Ridge Regression,1832.9921,5757523.0401,2398.8006,0.2569,3.7740,0.3636,0.0230
lar,Least Angle Regression,1833.1837,5757522.3816,2398.7999,0.2569,3.7743,0.3640,0.0160
llar,Lasso Least Angle Regression,1832.5988,5758079.6517,2398.9212,0.2568,3.7738,0.3625,0.0160
lr,Linear Regression,1833.3097,5758022.6368,2398.9024,0.2568,3.7743,0.3640,0.0380
